In [1]:
%matplotlib inline
from model import *
from agents import *
from globals import *
# from server import *
from schedule import *
from utility import *
from SALib.sample import saltelli
from mesa.batchrunner import BatchRunner, BatchRunnerMP
from SALib.analyze import sobol
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations
from IPython.display import clear_output
import multiprocessing as mp

In [ ]:
from mesa.batchrunner import BatchRunner, BatchRunnerMP

fixed_params = {"N": 1000,
                "network_type": 2,
                "no_of_neighbors": 3,
                "beta_component": 0.15,
                "social_influence": 0.06,
                "swingers": 3,
#                 "malicious_N":0,
                "echo_limit":0.95,
                "all_majority": False,
                "similarity_treshold": 0.16,
#                 "malicious_N": 0,
                 
               }
variable_params = {"opinions":  list(range(2, 6)),
                "malicious_N":list(range(0, 6)),
                  }

batch_run = BatchRunnerMP(Network,
                        iterations=30,
                        max_steps=100,
                        fixed_parameters=fixed_params,
                        variable_parameters=variable_params,
#                         model_reporters=model_reporters, 
                        nr_processes=4,
                        model_reporters={                
                            "radical_opinions": compute_radical_opinions,
                            "community_no": community_no,
#                             "echo_chambers": echo_no,
                            "majority_opinion": compute_majority_opinions,
                            "silent_spiral": compute_silent_spiral,
                            "compute_transitivity":compute_transitivity,
                            "compute_echo_chamber":compute_echo_chamber,
                            "echochamber_size":echochamber_size,
                            "echochamber_count":echochamber_count
                        })
batch_run.run_all()
data = batch_run.get_model_vars_dataframe()

#print data
print(data)

64it [19:51, 21.29s/it]